In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.craigslist_db
collection = db.items

In [4]:
# URL of page to be scraped
url = 'https://newjersey.craigslist.org/search/sss?sort=rel&query=guitar'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [7]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('li', class_='result-row')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_='result-title').text
        # Identify and return price of listing
        price = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (title and price and link):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

In [6]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5fbfb640237bd8c7d4848c67'), 'title': "Guild D55 Acoustic Guitar 3-Tone Suburst Mid 2000's **MINT**", 'price': '$2,999', 'url': 'https://newjersey.craigslist.org/msg/d/white-plains-guild-d55-acoustic-guitar/7237174059.html'}
{'_id': ObjectId('5fbfb641237bd8c7d4848c68'), 'title': 'FS/FT - Yamaha FG830 Acoustic Guitar', 'price': '$250', 'url': 'https://newjersey.craigslist.org/msg/d/mountainside-fs-ft-yamaha-fg830/7237150127.html'}
{'_id': ObjectId('5fbfb641237bd8c7d4848c69'), 'title': '12" EMINENCE GUITAR AMP SPEAKER', 'price': '$130', 'url': 'https://newjersey.craigslist.org/ele/d/caldwell-12-eminence-guitar-amp-speaker/7237138745.html'}
{'_id': ObjectId('5fbfb641237bd8c7d4848c6a'), 'title': 'Esteban Electric Acoustic Guitar ALC-200', 'price': '$200', 'url': 'https://newjersey.craigslist.org/msg/d/saddle-brook-esteban-electric-acoustic/7228386281.html'}
{'_id': ObjectId('5fbfb641237bd8c7d4848c6b'), 'title': 'PRO GUITAR SETUPS!', 'price': '$0', 'url': 'https://newjersey